<a href="https://colab.research.google.com/github/MadhuvanthiThanikachalam/Scrapping/blob/main/Web%20scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install requests_html
!pip3 install ecommercetools

In [ ]:
import requests
import urllib
from urllib.parse import urlparse
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession
from ecommercetools import seo

In [ ]:
pd.set_option('max_rows', 1000)
pd.set_option('max_colwidth', 1000)


In [ ]:
df = seo.get_sitemap("https://www.practicaldatascience.co.uk/sitemap.xml")
df = df[['loc']].head(100)
df.head()

,loc
0,https://practicaldatascience.co.uk/data-science/how-to-create-a-python-virtual-environment-for-jupyter
1,https://practicaldatascience.co.uk/data-science/how-to-engineer-date-features-using-pandas
2,https://practicaldatascience.co.uk/machine-learning/how-to-impute-missing-numeric-values-in-your-dataset
3,https://practicaldatascience.co.uk/machine-learning/how-to-interpret-the-confusion-matrix
4,https://practicaldatascience.co.uk/machine-learning/how-to-use-mean-encoding-in-your-machine-learning-models


In [ ]:
def get_source(url):
    """Return the source code for the provided URL. 

    Args: 
        url (string): URL of the page to scrape.

    Returns:
        response (object): HTTP response object from requests_html. 
    """

    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)

In [ ]:
def get_title(response):
    return response.html.find('title', first=True).text

In [ ]:
def get_post_links(response):
    return response.html.find('.article-post a')

In [ ]:
def is_internal(url, domain):
    if not bool(urlparse(url).netloc):
        return True
    elif url.startswith(domain):
        return True
    else:
        return False

In [ ]:
def scrape_links(df, url_column, domain):
    
    df_pages = pd.DataFrame(columns = ['url', 'title', 'post_link_href', 'post_link_text', 'internal'])
    
    for index, row in df.iterrows():
        response = get_source(row[url_column])
        data = {}
        data['url'] = row[url_column]
        data['title'] = get_title(response)
        
        post_links = get_post_links(response)
        if post_links:
            for link in post_links:                
                data['post_link_href'] = link.attrs['href']
                data['post_link_text'] = link.text
                data['internal'] = is_internal(link.attrs['href'], domain)
                df_pages = df_pages.append(data, ignore_index=True)
        else:
            df_pages = df_pages.append(data, ignore_index=True)   
    
    return df_pages

In [ ]:
df_pages = scrape_links(df, 'loc', 'https://practicaldatascience.co.uk')
df_pages.head(10)

,url,title,post_link_href,post_link_text,internal
0,https://practicaldatascience.co.uk/data-science/how-to-create-a-python-virtual-environment-for-jupyter,How to create a Python virtual environment for Jupyter,/data-science/how-to-build-a-data-science-workstation,Ubuntu data science workstation,True
1,https://practicaldatascience.co.uk/data-science/how-to-engineer-date-features-using-pandas,How to engineer date features using Pandas,https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects,Pandas documentation,False
2,https://practicaldatascience.co.uk/machine-learning/how-to-impute-missing-numeric-values-in-your-dataset,How to impute missing numeric values in your dataset,NaN,NaN,NaN
3,https://practicaldatascience.co.uk/machine-learning/how-to-interpret-the-confusion-matrix,How to interpret the confusion matrix,NaN,NaN,NaN
4,https://practicaldatascience.co.uk/machine-learning/how-to-use-mean-encoding-in-your-machine-learning-models,How to use mean encoding in your machine learning models,NaN,NaN,NaN
5,https://practicaldatascience.co.uk/data-science/how-to-use-python-regular-expressions-to-extract-information,How to use Python regular expressions to extract information,NaN,NaN,NaN
6,https://practicaldatascience.co.uk/data-science/how-to-use-scikit-learn-datasets-in-data-science-projects,How to use scikit-learn datasets in data science projects,https://scikit-learn.org/stable/datasets/toy_dataset.html#boston-house-prices-dataset,Boston house prices dataset,False
7,https://practicaldatascience.co.uk/data-science/how-to-use-scikit-learn-datasets-in-data-science-projects,How to use scikit-learn datasets in data science projects,https://scikit-learn.org/stable/datasets/toy_dataset.html#iris-plants-dataset,Iris dataset,False
8,https://practicaldatascience.co.uk/data-science/how-to-use-scikit-learn-datasets-in-data-science-projects,How to use scikit-learn datasets in data science projects,https://scikit-learn.org/stable/datasets/toy_dataset.html#diabetes-dataset,diabetes dataset,False
9,https://practicaldatascience.co.uk/data-science/how-to-use-scikit-learn-datasets-in-data-science-projects,How to use scikit-learn datasets in data science projects,https://scikit-learn.org/stable/datasets/toy_dataset.html,optical recognition of handwritten digits dataset,False


In [ ]:
df_internal = df_pages[df_pages['internal']==True]
df_internal.head()

,url,title,post_link_href,post_link_text,internal
0,https://practicaldatascience.co.uk/data-science/how-to-create-a-python-virtual-environment-for-jupyter,How to create a Python virtual environment for Jupyter,/data-science/how-to-build-a-data-science-workstation,Ubuntu data science workstation,True
16,https://practicaldatascience.co.uk/data-science/how-to-access-google-analytics-data-in-pandas-using-gapandas,How to use GAPandas to view your Google Analytics data,/tag/google-analytics/,Google Analytics,True
18,https://practicaldatascience.co.uk/data-science/how-to-access-google-analytics-data-in-pandas-using-gapandas,How to use GAPandas to view your Google Analytics data,/data-science/how-to-create-a-python-virtual-environment-for-jupyter,Jupyter notebooks,True
45,https://practicaldatascience.co.uk/machine-learning/a-quick-guide-to-machine-learning-uplift-models,A quick guide to machine learning uplift models,/data-science/a-quick-guide-to-customer-segmentation,customer segmentation,True
50,https://practicaldatascience.co.uk/machine-learning/a-quick-guide-to-machine-learning,A quick guide to machine learning,/machine-learning/how-to-use-recursive-feature-elimination-in-your-models,dimensionality reduction,True


In [ ]:
df_internal['url'].count()

48

In [ ]:
df_internal['url'].nunique()

20

In [ ]:
df_external = df_pages[df_pages['internal']==False]
df_external.head()

,url,title,post_link_href,post_link_text,internal
1,https://practicaldatascience.co.uk/data-science/how-to-engineer-date-features-using-pandas,How to engineer date features using Pandas,https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects,Pandas documentation,False
6,https://practicaldatascience.co.uk/data-science/how-to-use-scikit-learn-datasets-in-data-science-projects,How to use scikit-learn datasets in data science projects,https://scikit-learn.org/stable/datasets/toy_dataset.html#boston-house-prices-dataset,Boston house prices dataset,False
7,https://practicaldatascience.co.uk/data-science/how-to-use-scikit-learn-datasets-in-data-science-projects,How to use scikit-learn datasets in data science projects,https://scikit-learn.org/stable/datasets/toy_dataset.html#iris-plants-dataset,Iris dataset,False
8,https://practicaldatascience.co.uk/data-science/how-to-use-scikit-learn-datasets-in-data-science-projects,How to use scikit-learn datasets in data science projects,https://scikit-learn.org/stable/datasets/toy_dataset.html#diabetes-dataset,diabetes dataset,False
9,https://practicaldatascience.co.uk/data-science/how-to-use-scikit-learn-datasets-in-data-science-projects,How to use scikit-learn datasets in data science projects,https://scikit-learn.org/stable/datasets/toy_dataset.html,optical recognition of handwritten digits dataset,False


In [ ]:
df_external['url'].count()

123

In [ ]:
df_external['url'].nunique()

37

In [ ]:
df_no_internal_links = df_pages[df_pages['internal']!=True]
df_no_internal_links.head()

,url,title,post_link_href,post_link_text,internal
1,https://practicaldatascience.co.uk/data-science/how-to-engineer-date-features-using-pandas,How to engineer date features using Pandas,https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects,Pandas documentation,False
2,https://practicaldatascience.co.uk/machine-learning/how-to-impute-missing-numeric-values-in-your-dataset,How to impute missing numeric values in your dataset,NaN,NaN,NaN
3,https://practicaldatascience.co.uk/machine-learning/how-to-interpret-the-confusion-matrix,How to interpret the confusion matrix,NaN,NaN,NaN
4,https://practicaldatascience.co.uk/machine-learning/how-to-use-mean-encoding-in-your-machine-learning-models,How to use mean encoding in your machine learning models,NaN,NaN,NaN
5,https://practicaldatascience.co.uk/data-science/how-to-use-python-regular-expressions-to-extract-information,How to use Python regular expressions to extract information,NaN,NaN,NaN


In [ ]:
df_no_internal_links['url'].nunique()

89

In [ ]:
df_no_links = df_pages[df_pages['internal'].isnull()]
df_no_links.head()

,url,title,post_link_href,post_link_text,internal
2,https://practicaldatascience.co.uk/machine-learning/how-to-impute-missing-numeric-values-in-your-dataset,How to impute missing numeric values in your dataset,NaN,NaN,NaN
3,https://practicaldatascience.co.uk/machine-learning/how-to-interpret-the-confusion-matrix,How to interpret the confusion matrix,NaN,NaN,NaN
4,https://practicaldatascience.co.uk/machine-learning/how-to-use-mean-encoding-in-your-machine-learning-models,How to use mean encoding in your machine learning models,NaN,NaN,NaN
5,https://practicaldatascience.co.uk/data-science/how-to-use-python-regular-expressions-to-extract-information,How to use Python regular expressions to extract information,NaN,NaN,NaN
28,https://practicaldatascience.co.uk/data-science/how-to-create-a-pandas-dataframe,How to create a Pandas dataframe,NaN,NaN,NaN


In [ ]:
import re
import requests
import requests.exceptions
from urllib.parse import urlsplit
from collections import deque
from bs4 import BeautifulSoup

# starting url. replace google with your own url.
starting_url = 'https://practicaldatascience.co.uk'

# a queue of urls to be crawled
unprocessed_urls = deque([starting_url])

# set of already crawled urls for email
processed_urls = set()

# a set of fetched emails
emails = set()

# process urls one by one from unprocessed_url queue until queue is empty
while len(unprocessed_urls):

    # move next url from the queue to the set of processed urls
    url = unprocessed_urls.popleft()
    processed_urls.add(url)

    # extract base url to resolve relative links
    parts = urlsplit(url)
    base_url = "{0.scheme}://{0.netloc}".format(parts)
    path = url[:url.rfind('/')+1] if '/' in parts.path else url

    # get url's content
    print("Crawling URL %s" % url)
    try:
        response = requests.get(url)
    except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
        # ignore pages with errors and continue with next url
        continue

    # extract all email addresses and add them into the resulting set
    # You may edit the regular expression as per your requirement
    new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", response.text, re.I))
    emails.update(new_emails)
    print(emails)

    mobile = re.findall(r'(?:\+?\d{2}[ -]?)?\d{10}', response.text)
    print("Ph no: ",mobile)
    # create a beutiful soup for the html document
    soup = BeautifulSoup(response.text, 'lxml')

    # Once this document is parsed and processed, now find and process all the anchors i.e. linked urls in this document
    for anchor in soup.find_all("a"):
        # extract link url from the anchor
        link = anchor.attrs["href"] if "href" in anchor.attrs else ''
        # resolve relative links (starting with /)
        if link.startswith('/'):
            link = base_url + link
       
        # add the new url to the queue if it was not in unprocessed list nor in processed list yet
        if not link in unprocessed_urls and not link in processed_urls:
            unprocessed_urls.append(link)


Crawling URL https://practicaldatascience.co.uk
set()
Ph no:  []
Crawling URL https://practicaldatascience.co.uk/
set()
Ph no:  []
Crawling URL https://practicaldatascience.co.uk/data-science
set()
Ph no:  ['7277865394', '8537762300', '2871816479', '0316032335', '965787244964', '191370639683', '191370639683', '1097240788', '7830261343', '1001488076', '4271441634', '0115348459', '0558799533', '551947277230']
Crawling URL https://practicaldatascience.co.uk/machine-learning
set()
Ph no:  ['726999206882', '6942964256', '085988586754', '250285870171', '191714745265']
Crawling URL https://practicaldatascience.co.uk/data-engineering
set()
Ph no:  ['242720995660', '0716272353']
Crawling URL https://practicaldatascience.co.uk/data-science/data-science-courses-for-budding-data-scientists
set()
Ph no:  ['191370639683', '191714745265', '7277865394']
Crawling URL https://practicaldatascience.co.uk/data-science/a-quick-guide-to-customer-segmentation
set()
Ph no:  ['0316032335', '2639227395', '853776